# Learning Functional ANOVA Decompositions in FAVA: Continuous Responses

In this tutorial, we show how to solve penalized least squares problems of the form

$$\hat{f} = \arg \min_{f \in H} \sum_{n=1}^N (y^{(n)} - f(x^{(n)}))^2 + \lambda \|f\|_{\mathcal{H}}^2, $$

where $\mathcal{H}$ consists of the space of interactions of order up to $Q$: 

$$\mathcal{H} = \left\{f: f = \sum_{V: |V| \leq Q} f_V \right\} \quad \text{s.t.} \quad \forall A \subsetneq B, \ f_A \perp_{\mathcal{H}} f_B.$$

# Import packages

We start by importing the required packages in `fava` and `jax`.

In [1]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
import jax

from fava.inference.fit import GaussianSKIMFA
from fava.basis.maps import LinearBasis
from fava.misc.scheduler import truncScheduler, constantScheduler
from fava.misc.logger import GausLogger
from fava.decomposers.decomposer import all_subsets
from fava.decomposers.tensor_product import TensorProductKernelANOVA, LinearANOVA

# Simulating data

Below we generate simulated data, where the covariates $x \in \mathbb{R}^{100}$ and $y \in \mathbb{R}$. We assume that the first 4 covariates drive the response $y$, and the remaining covariates have no influence on the response. Specifically, the $n$th datapoint is generated as follows: $y^{(n)} = x_1^{(n)}x_2^{(n)} + x_2^{(n)}x_3^{(n)} + \epsilon^{(n)}$, where $\epsilon^{(n)} \overset{\text{i.i.d.}}{\sim} \mathcal{N}(0, 1)$ for $1 \leq n \leq 500$.


In [2]:
key = random.PRNGKey(0) # set seed
N = 500
p = 100
X = random.normal(key, shape=(N, p))
epsilon = random.normal(key, shape=(N, ))
Y = X[:, 0] + X[:, 1] + X[:, 2] * X[:, 3] + epsilon

# Split data into training and validation

In [3]:
# Use 80% of the data for training and 20% for validation
X_train = X[:400, :]
Y_train = Y[:400]

X_valid = X[400:, :]
Y_valid = Y[400:]

# Fit SKIM-FA Model

Below we initialize SKIM-FA hyperparameters with default values. We run a total of $T=500$ iterations. The parameters that maximize predictive performance on the validation set are selected.

In [4]:
kernel_params = dict()
Q = 2
kernel_params['U_tilde'] = jnp.ones(p)
kernel_params['eta'] = jnp.ones(Q+1)

hyperparams = dict()
hyperparams['sigma_sq'] = .5 * jnp.var(Y)
hyperparams['c'] = .2

opt_params = dict()
opt_params['cg'] = True
opt_params['cg_tol'] = .01
opt_params['M'] = 100
opt_params['gamma'] = .1
opt_params['T'] = 500
opt_params['scheduler'] = constantScheduler() # we won't get exact sparsity since c is constant
featprocessor = LinearBasis(X_train)
logger = GausLogger(100)

skim = GaussianSKIMFA(X_train, Y_train, X_valid, Y_valid, featprocessor)

skim.fit(key, hyperparams, kernel_params, opt_params, 
            logger=GausLogger())

  0%|                                                                                                                 | 0/500 [00:00<?, ?it/s]

============================== Iteration 0/500 ==============================
There are 100 covariates selected.


  0%|▏                                                                                                        | 1/500 [00:04<34:53,  4.19s/it]

MSE (Validation)=2.5803.
R2 (Validation)=0.4308.
eta=[1.0000176  1.0927966  0.90706474]
c=0.2
kappa=[0.427512   0.43775743 0.38954645 0.3935901  0.37155294 0.37196064
 0.37526184 0.37408942 0.37568456 0.37432647 0.36845684 0.3702034
 0.37719196 0.38341844 0.37717736 0.36670595 0.3718865  0.37691826
 0.373253   0.37422097 0.36725998 0.37259445 0.37028933 0.3774687
 0.3699179  0.37486166 0.36752617 0.37550277 0.37107378 0.37370145
 0.37118727 0.37558192 0.3727138  0.37480414 0.3790542  0.36897695
 0.36873615 0.37040257 0.3758871  0.3640982  0.37777245 0.37394568
 0.3741271  0.37913078 0.36967307 0.37678212 0.36617234 0.37765032
 0.37358582 0.37470585 0.37127936 0.3739831  0.37635857 0.37284583
 0.37119448 0.36952233 0.37866533 0.37040457 0.36523217 0.37135786
 0.37115157 0.37187934 0.37034845 0.38416728 0.37458822 0.3745625
 0.3640228  0.36721563 0.37601233 0.36932468 0.37321633 0.37741148
 0.37234172 0.36836708 0.38273335 0.36760545 0.3676412  0.37340626
 0.370626   0.368869   0.3669037

 20%|████████████████████▌                                                                                  | 100/500 [00:42<02:36,  2.56it/s]

...

 80%|██████████████████████████████████████████████████████████████████████████████████▍                    | 400/500 [02:40<00:38,  2.59it/s]

============================== Iteration 400/500 ==============================
There are 100 covariates selected.


 80%|██████████████████████████████████████████████████████████████████████████████████▌                    | 401/500 [02:41<00:46,  2.12it/s]

MSE (Validation)=2.0149.
R2 (Validation)=0.5555.
eta=[1.0111151 1.7499819 1.2703197]
c=0.2
kappa=[6.22618318e-01 7.54343927e-01 7.78473616e-01 6.33080244e-01
 1.86264515e-07 6.03497028e-06 4.04189974e-01 2.88623840e-01
 3.21125239e-01 4.21404839e-04 1.86264515e-07 6.22030348e-04
 3.95276695e-01 4.42406893e-01 2.69092739e-01 6.92164525e-03
 1.74995512e-04 1.04006886e-01 4.86630559e-01 4.17416453e-01
 1.41881615e-01 6.26288295e-01 4.47485596e-02 1.83656812e-05
 2.08616257e-06 2.63900399e-01 1.76951289e-06 7.54447430e-02
 2.79396772e-07 4.61198390e-03 3.65673751e-01 5.37077904e-01
 6.86571002e-05 1.13621354e-06 5.06975651e-01 1.30385160e-07
 2.02107593e-01 5.02914190e-07 4.28408384e-07 6.92631453e-02
 2.42169261e-01 7.87873417e-02 2.79396772e-07 5.38698971e-01
 1.78796187e-01 2.92907298e-01 1.26659870e-06 1.05669707e-01
 4.65661287e-07 3.32516283e-01 5.53578138e-05 9.00216401e-05
 5.41176915e-01 1.74921006e-04 1.26398489e-01 1.97207689e-01
 1.07161835e-01 2.84592241e-01 4.48504463e-03 6.1

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [03:19<00:00,  2.50it/s]


# Compare predictive performance against random forest

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import numpy as np

In [6]:
rf = RandomForestRegressor(n_estimators=1000, random_state=0)
rf.fit(np.array(X), np.array(Y))

RandomForestRegressor(n_estimators=1000, random_state=0)

In [7]:
# Generate new test set and compare R^2 performance
N_test = 1000
X_test = random.normal(key, shape=(N_test, p))
epsilon = random.normal(key, shape=(N_test, ))
Y_test = X_test[:, 0] + X_test[:, 1] + X_test[:, 2] * X_test[:, 3] + epsilon

In [8]:
rf_r2 = rf.score(np.array(X_test), np.array(Y_test))
skimfa_r2 = r2_score(Y_test, skim.predict(X_test))

In [9]:
print(f'Random Forest Test R^2: {round(rf_r2, 4)}')
print(f'SKIM-FA Test R^2: {round(skimfa_r2, 4)}')

Random Forest Test R^2: 0.4495
SKIM-FA Test R^2: 0.6038


# Report effects

Now we compute all of the main/interaction effects found by SKIM-FA. Since we fit a linear interaction model, we report the regression coefficients associated with each effect. Since we used a constant scheduler for c, we do not get exact sparsity.

In [10]:
# Compute all main and in
lanova = LinearANOVA(skim)
V_all = all_subsets(skim.selected_covariates(), 2, True)
all_skim_effects = dict()
for V in V_all:
    all_skim_effects[V] = lanova.get_coef(V)

In [11]:
# Report Top 5 positive effects found
top5_positive = sorted(all_skim_effects, key=all_skim_effects.get, reverse=True)[:5]
top5_negative = sorted(all_skim_effects, key=all_skim_effects.get, reverse=False)[:5]

print('Top 5 Strongest Positive Effects Found:')
for V in top5_positive:
    print(V,  round(all_skim_effects[V], 3))

Top 5 Strongest Positive Effects Found:
(1,) 0.923
(0,) 0.861
(2, 3) 0.739
(65,) 0.109
(30,) 0.105


In [12]:
print('Top 5 Strongest Negative Effects Found:')
for V in top5_negative:
    print(V,  round(all_skim_effects[V], 3))

Top 5 Strongest Negative Effects Found:
(2,) -0.107
(52,) -0.091
(2, 21) -0.089
(83,) -0.088
(70,) -0.086


In [13]:
Y_test_pred = skim.predict(X_test)
variation_intercept = lanova.get_variation_at_order(X_test, 0)
variation_additive = lanova.get_variation_at_order(X_test, 1)
variation_pairwise = lanova.get_variation_at_order(X_test, 2)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 4950/4950 [00:29<00:00, 170.24it/s]


In [15]:
check = variation_intercept + variation_additive + variation_pairwise

In [22]:
assert jnp.abs(Y_test_pred - check).max() < 1e-5

In [27]:
variation_additive.

DeviceArray([-1.31695580e+00,  1.61611164e+00, -1.19587764e-01,
             -1.09929502e+00,  2.70619678e+00, -8.42398345e-01,
             -4.97205168e-01, -7.42296636e-01, -1.02377415e+00,
              2.34906226e-01, -1.50630578e-01, -5.91090322e-01,
              2.45717120e+00, -1.63255095e+00, -1.41911674e+00,
              7.78063834e-01,  3.65272313e-01,  2.70254445e+00,
              2.33002111e-01, -2.50812531e+00,  1.26681304e+00,
              2.51954913e-01, -2.89740300e+00,  7.23200679e-01,
             -8.01900387e-01,  1.50185740e+00,  1.68301836e-02,
              2.93846488e+00, -1.83273602e+00,  9.19020697e-02,
              4.25252356e-02,  1.04629314e+00,  2.18129659e+00,
             -6.67625815e-02,  9.00431037e-01,  1.85603350e-01,
              1.42799163e+00,  1.01007320e-01,  2.05034542e+00,
             -1.26787341e+00,  3.32559422e-02,  1.84117627e+00,
              1.61477172e+00, -1.68884814e+00, -8.96534920e-01,
             -5.05311310e-01,  4.7241829

In [21]:
Y_test_pred

DeviceArray([-3.23225260e-02,  1.18766284e+00,  6.09884620e-01,
             -2.28547215e+00,  8.71474981e-01, -1.66278851e+00,
              1.63102007e+00, -1.01750422e+00, -6.43783152e-01,
             -4.82864857e-01,  2.78809845e-01,  1.95042133e-01,
              2.12673354e+00, -2.94063830e+00, -2.01552868e+00,
              7.09180474e-01, -1.71351194e-01,  2.74175167e+00,
             -1.22834444e+00, -1.64807272e+00,  6.79801762e-01,
              6.40606880e-02, -1.75504804e+00,  2.80366838e-02,
              8.32024693e-01,  1.27894759e+00, -4.82792139e-01,
              1.08192515e+00, -1.79151797e+00,  1.08313560e-01,
             -4.10003722e-01,  3.66284609e-01,  1.64024568e+00,
              1.04712629e+00,  2.90388179e+00,  2.47578454e+00,
              1.19321287e+00,  7.44124711e-01,  1.77179968e+00,
             -3.94072056e-01,  4.35680270e-01,  1.43238294e+00,
              1.24127936e+00, -1.55620325e+00, -4.97299528e+00,
              1.31685328e+00, -5.6056052

In [18]:
all_skim_effects

{(): 0.008750999346375465,
 (0,): 1.0433865785598755,
 (1,): 0.9432764053344727,
 (2,): -0.10295327752828598,
 (3,): 0.033366285264492035,
 (4,): -0.09841124713420868,
 (5,): 0.006347221322357655,
 (6,): -0.05333470553159714,
 (7,): -0.03528089076280594,
 (8,): 0.10618141293525696,
 (9,): -0.1582774668931961,
 (10,): -0.007712193299084902,
 (11,): -0.016395868733525276,
 (12,): -0.1322556883096695,
 (13,): 0.018340393900871277,
 (14,): -0.14529678225517273,
 (15,): -0.058393288403749466,
 (16,): -0.03898949548602104,
 (17,): -0.2872205376625061,
 (18,): -0.10594488680362701,
 (19,): 0.12228113412857056,
 (20,): 0.17517194151878357,
 (21,): -0.08068910986185074,
 (22,): 0.037124283611774445,
 (23,): 0.20786164700984955,
 (24,): -0.04180143401026726,
 (25,): -0.12091130018234253,
 (26,): 0.0017131451750174165,
 (27,): -0.15818792581558228,
 (28,): 0.007069380488246679,
 (29,): -0.03701794147491455,
 (30,): 0.18012690544128418,
 (31,): -0.061385203152894974,
 (32,): -0.06541499495506287,


In [25]:
X_test[:, 0] + X_test[:, 1]

DeviceArray([-1.01255631e+00,  1.21531248e+00, -2.26160467e-01,
             -2.34571600e+00,  1.10745716e+00,  6.19640350e-02,
              1.06760979e-01, -5.13819456e-01, -6.13765478e-01,
              1.08492732e-01,  6.51032090e-01,  1.67322218e-01,
              2.63528728e+00, -1.71394277e+00, -2.49961782e+00,
              1.95496023e-01, -1.32662666e+00,  2.80388403e+00,
              2.57455677e-01, -2.20552325e+00,  4.26900014e-03,
              7.25513697e-01, -1.09891033e+00, -2.17479795e-01,
              1.59804773e+00,  4.51631010e-01,  4.42312002e-01,
              1.39190364e+00, -1.33168900e+00,  1.39957070e-02,
             -5.90180278e-01, -1.05186117e+00,  1.11463583e+00,
              2.57611275e-03,  5.32105029e-01,  8.54863405e-01,
              1.66246450e+00, -5.66894352e-01,  9.63153780e-01,
             -1.93817890e+00,  1.52129197e+00,  2.57087159e+00,
              1.54113758e+00, -1.40582728e+00, -1.38507581e+00,
             -1.30817056e-01,  4.0361467

In [31]:
(X_test[:, 2] * X_test[:, 3])

DeviceArray([ 1.52120411e+00, -6.82581842e-01,  2.43750975e-01,
             -1.03522623e+00, -4.82180446e-01, -2.35352492e+00,
             -3.66908342e-01, -5.41399717e-02,  6.83009997e-02,
              3.21155101e-01, -1.27785131e-01, -1.87579393e-01,
              3.83262277e-01, -8.48824918e-01,  8.00536200e-02,
             -2.97299504e-01, -8.09949785e-02,  2.36237854e-01,
             -4.69085395e-01,  7.04596043e-01,  1.10533275e-01,
             -1.61302865e+00,  7.73017034e-02, -4.50229108e-01,
             -4.84288454e-01, -1.05414219e-01, -5.96565187e-01,
              3.29071714e-04, -1.60371196e+00, -4.89815891e-01,
             -1.07617220e-02, -1.09506138e-01, -2.24987611e-01,
              7.49990106e-01,  1.72468030e+00,  2.38404250e+00,
             -8.34121406e-01,  1.53158629e+00,  1.38413504e-01,
              1.24883461e+00, -2.16618422e-02, -1.72553614e-01,
             -5.77715099e-01,  5.82536101e-01, -5.94747257e+00,
              1.93065643e-01, -1.1707055

In [36]:
jnp.product(X_test[:, jnp.array([2, 3])], axis=1)

DeviceArray([ 1.52120411e+00, -6.82581842e-01,  2.43750975e-01,
             -1.03522623e+00, -4.82180446e-01, -2.35352492e+00,
             -3.66908342e-01, -5.41399717e-02,  6.83009997e-02,
              3.21155101e-01, -1.27785131e-01, -1.87579393e-01,
              3.83262277e-01, -8.48824918e-01,  8.00536200e-02,
             -2.97299504e-01, -8.09949785e-02,  2.36237854e-01,
             -4.69085395e-01,  7.04596043e-01,  1.10533275e-01,
             -1.61302865e+00,  7.73017034e-02, -4.50229108e-01,
             -4.84288454e-01, -1.05414219e-01, -5.96565187e-01,
              3.29071714e-04, -1.60371196e+00, -4.89815891e-01,
             -1.07617220e-02, -1.09506138e-01, -2.24987611e-01,
              7.49990106e-01,  1.72468030e+00,  2.38404250e+00,
             -8.34121406e-01,  1.53158629e+00,  1.38413504e-01,
              1.24883461e+00, -2.16618422e-02, -1.72553614e-01,
             -5.77715099e-01,  5.82536101e-01, -5.94747257e+00,
              1.93065643e-01, -1.1707055

In [38]:
tanova = TensorProductKernelANOVA(skim)
tall_skim_effects = dict()
for V in V_all:
    tall_skim_effects[V] = tanova.get_effect(skim.featprocessor.transform(X_test),V)

In [50]:
jnp.array(list(tall_skim_effects.values())).sum(axis=0)

DeviceArray([-3.32765222e+00,  3.23535490e+00,  7.42995024e+00,
             -8.66494834e-01,  2.56822395e+00, -3.90953541e+00,
              3.25818038e+00, -4.81084490e+00, -2.39255643e+00,
             -4.18491220e+00, -3.29074383e+00,  2.91220140e+00,
              2.35722971e+00, -5.43159723e+00, -2.77370811e+00,
              4.01422882e+00,  4.13087940e+00,  5.35396099e+00,
             -8.11057854e+00, -2.72536063e+00,  2.75264311e+00,
             -6.29099250e-01, -1.15897799e+00,  2.28632259e+00,
              9.82399940e-01,  3.00167537e+00, -3.38791943e+00,
             -1.28489780e+00,  7.26147175e-01,  1.71995020e+00,
             -3.43963718e+00,  2.42093325e+00,  6.04936838e+00,
              1.22776723e+00,  6.35369444e+00,  6.32842779e+00,
              5.58356524e+00,  2.13614845e+00,  2.35008025e+00,
              1.51247537e+00,  4.89617586e-01,  1.34604001e+00,
              2.21819210e+00, -4.70656776e+00, -6.24030972e+00,
              5.96039486e+00, -1.9103732